<a href="https://colab.research.google.com/github/dimitrod/ehu_nlp_dimathina/blob/Evaluation/Evaluation_Chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!git clone https://github.com/mandarjoshi90/triviaqa.git

fatal: destination path 'triviaqa' already exists and is not an empty directory.


In [19]:
%cd triviaqa

/content/triviaqa


In [20]:
!pip install -r requirements.txt

In [21]:
import subprocess
import os
import json
from tqdm import tqdm
import importlib
import argparse

In [22]:
env = os.environ.copy()
env["PYTHONPATH"] = "/content/triviaqa"

In [23]:
%cd ../model

/content/model


In [25]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.7/409.7 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.21
    Uninstalling langchain-core-0.3.21:
      Successfully uninstalled langchain-core-0.3.21
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.9
    Uninstalling langchain-0.3.9:
      Successfully uninstalled langchain-0.3.9


In [26]:
%cd ..

/content


In [27]:
def evaluate_triviaqa(dataset_file, prediction_file):
  script_path = "/content/triviaqa/evaluation/triviaqa_evaluation.py"
  result = subprocess.run([
    "python", script_path,
    "--dataset_file", dataset_file,
    "--prediction_file", prediction_file],
    env=env,
    capture_output=True,
    text=True,
  )
  print(result.stdout)

In [28]:
def get_answers(directory, questions, model):
    answers = {}

    for i in tqdm(range(len(questions)), desc="Retrieving Answers"):
        question = questions[i]
        answers[question[0]] = model.invoke(question[1])
    write_data(directory + "/predictions.json", answers)

In [29]:
def load_questions(path, split):
    create_split(path, split)

    with open(f"{path}/evaluation_dataset.json", "r", encoding="utf-8") as f:
        data = json.loads(f.read())

    questions = []

    for i in tqdm(range(len(data["Data"])),desc="Loading Questions"):
        question = data["Data"][i]
        questions.append([question["QuestionId"], question["Question"]])
    return questions

In [30]:
def create_split(path, split):
  with open(f"{path}/original_dataset.json", "r", encoding="utf-8") as f:
    data = json.loads(f.read())

  data["Data"] = data["Data"][:split]

  write_data(f"{path}/evaluation_dataset.json", data)

In [35]:
def load_model(path, model_name):
  print("Loading model...")
  module = importlib.import_module(path)
  model = getattr(module, model_name)
  return model()

In [32]:
def write_data(path, data):
  with open(path, "w", encoding="utf-8") as f:
    f.write(json.dumps(data))

In [33]:
def get_args():
    parser = argparse.ArgumentParser(
        description='Evaluation Chain for TriviaQA {}')
    parser.add_argument('--dataset_file', help='Dataset file')
    parser.add_argument('--model_file', help='Model file')
    args = parser.parse_args()
    return args

In [34]:
if __name__ == "__main__":
    #args = get_args()
    directory = "/content/evaluation_data"
    split = 100
    model_path = "model.rag_qa_embeddings"
    model_name = "rag_qa_embeddings"

    questions = load_questions(directory, split)
    model = load_model(model_path, model_name)
    print("Model initiated")
    print("Starting QA...")
    answers = get_answers(directory, questions, model)
    print("QA complete")
    print("Test results:")
    evaluate_triviaqa(directory + "/evaluation_dataset.json", directory + "/predictions.json")


Loading Questions: 100%|██████████| 100/100 [00:00<00:00, 268865.64it/s]


Loading model...


/content/model/rag_qa_embeddings.py:27: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  return HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warning

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

RuntimeError: Error in faiss::FileIOReader::FileIOReader(const char*) at /project/faiss/faiss/impl/io.cpp:67: Error: 'f' failed: could not open /content/qa_database/index.faiss for reading: No such file or directory